# Pre-Processing

In [2]:
import os
import pandas as pd
import numpy as np

# Define the base directory
notebook_dir = os.getcwd()
base_dir = os.path.join(notebook_dir, 'Hourly Energy Consumption')

# Define utilities with file names
utilities = {
    'AEP': 'AEP_hourly.csv',
    'COMED': 'COMED_hourly.csv',
    'DAYTON': 'DAYTON_hourly.csv',
    'DEOK': 'DEOK_hourly.csv',
    'DOM': 'DOM_hourly.csv',
    'DUQ': 'DUQ_hourly.csv',
    'EKPC': 'EKPC_hourly.csv',
    'FE': 'FE_hourly.csv',
    'NI': 'Ni_hourly.csv',
    'PJMW': 'PJMW_hourly.csv'
}

# Function to remove outliers
def remove_outliers(series, threshold=3):
    mean = series.mean()
    std = series.std()
    return series[(series > mean - threshold * std) & (series < mean + threshold * std)]

# Function to handle duplicate timestamps
def handle_duplicates(df, mw_col):
    # Group by index and aggregate using mean
    df = df.groupby(df.index).mean()
    return df

# Preprocess function that loads data, handles outliers, duplicates, and adds time-based features
def preprocess_data(file_path, mw_col):
    df = pd.read_csv(file_path)
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    df.set_index('Datetime', inplace=True)

    # Handle duplicates
    df = handle_duplicates(df, mw_col=mw_col)

    # Remove outliers
    df[mw_col] = remove_outliers(df[mw_col])

    # Drop any remaining missing values
    df.dropna(inplace=True)

    # Add time-based features: hour, day_of_week, and month
    df['hour'] = df.index.hour
    df['day_of_week'] = df.index.dayofweek
    df['month'] = df.index.month

    return df

# Go over all utilities and preprocess the data
processed_data = {}
for utility, filename in utilities.items():
    file_path = os.path.join(base_dir, filename)
    df = preprocess_data(file_path, mw_col=f'{utility}_MW')
    if not df.empty:
        print(f"Preprocessed data for: {utility}")
    else:
        print(f"Skipping {utility} due to insufficient data.")
    processed_data[utility] = df


Preprocessed data for: AEP
Preprocessed data for: COMED
Preprocessed data for: DAYTON
Preprocessed data for: DEOK
Preprocessed data for: DOM
Preprocessed data for: DUQ
Preprocessed data for: EKPC
Preprocessed data for: FE
Preprocessed data for: NI
Preprocessed data for: PJMW


In [3]:
# Print data preprocessing results to confirm
for utility, df in processed_data.items():
    print(f"Preprocessed data for {utility}:")
    print(df.head())

Preprocessed data for AEP:
                      AEP_MW  hour  day_of_week  month
Datetime                                              
2004-10-01 01:00:00  12379.0     1            4     10
2004-10-01 02:00:00  11935.0     2            4     10
2004-10-01 03:00:00  11692.0     3            4     10
2004-10-01 04:00:00  11597.0     4            4     10
2004-10-01 05:00:00  11681.0     5            4     10
Preprocessed data for COMED:
                     COMED_MW  hour  day_of_week  month
Datetime                                               
2011-01-01 01:00:00    9631.0     1            5      1
2011-01-01 02:00:00    9273.0     2            5      1
2011-01-01 03:00:00    9011.0     3            5      1
2011-01-01 04:00:00    8741.0     4            5      1
2011-01-01 05:00:00    8694.0     5            5      1
Preprocessed data for DAYTON:
                     DAYTON_MW  hour  day_of_week  month
Datetime                                                
2004-10-01 01:00:00    